# Scikit Learn <a href="https://scikit-learn.org/stable/#"><img id='logo' height=36 src="https://scikit-learn.org/stable/_static/scikit-learn-logo-small.png"></a>
<style>
#logo {
    background-color: white;
    border-radius: 10px;
    }
</style>

scikit-learn (или sklearn) это библиотека на языке программирования Python, которая используется для решения задач машинного обучения, в том числе классификации, регрессии, кластеризации и обработки данных. Она предоставляет реализацию многих алгоритмов машинного обучения, таких как метод опорных векторов (SVM), случайный лес (Random Forest), метод главных компонент (PCA) и многие другие.

Библиотека scikit-learn позволяет быстро создавать и применять модели машинного обучения на больших наборах данных. Она также включает в себя множество функций для предобработки данных, оценки качества моделей и выбора оптимальных параметров моделей.

### Загрузка и визуализация данных 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

train_data = pd.read_csv('ice_cream_selling_train.csv')
train_data.head()

In [ ]:
X_train = train_data['temperature'].to_numpy()
y_train = train_data['ice_cream_sales'].to_numpy()
print(X_train.shape, y_train.shape)

In [ ]:
plt.scatter(X_train, y_train)
plt.ylim(0, 270)
plt.xlabel('temperature')
plt.ylabel('ice_cream_sales')

### Выбор модели и обучение 

In [ ]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()       # создается объект модели
# обучение модели на данных
model.fit(X_train.reshape(-1, 1), y_train)      

### Прогнозирование

In [ ]:
# загрузка тестовых данных и конвертация в массивы NumPy
test_data = pd.read_csv('ice_cream_selling_test.csv')
X_test = test_data['temperature'].to_numpy()
y_test = test_data['ice_cream_sales'].to_numpy()

# произведем прогнозирование на тестовых данных
y_test_predicted = model.predict(X_test.reshape(-1, 1))

header = f"X_test\ty_test\ty_test_predicted"
print(header, '\n', '-'*len(header), sep='')
for j in range(10):
    i = np.random.randint(0, len(y_test))
    print(f"{X_test[i]}\t{y_test[i]}\t{int(y_test_predicted[i])}")

In [ ]:
# произведем прогнозирование на тренировочных данных
y_train_predicted = model.predict(X_train.reshape(-1, 1))

plt.scatter(X_train, y_train, c='grey', s=15)
plt.scatter(X_test, y_test, c='green')
plt.plot(X_train, y_train_predicted)
plt.ylim(0, 270)

plt.xlabel('temperature')
plt.ylabel('ice_cream_sales')

### Оценка результата

Для того, чтобы оценить регрессионную модель, можно оценить среднеквадратическое отклонение, используюя функцию `mean_squared_error()` из модуля `metrics` библиотеки `sklearn`.

In [ ]:
from sklearn.metrics import mean_squared_error as mse

train_rmse = mse(y_train, y_train_predicted, squared=False)
test_rmse = mse(y_test, y_test_predicted, squared=False)

print(f'{train_rmse = }, {test_rmse = }')

**Среднеквадратическая ошибка**
$$
\mathrm{MSE} = \frac{1}{n} \sum_{i=1}^n (y_i - \hat{y_i} )^2 =  \frac{1}{n} \left[(y_1 - \hat{y_1})^2 + \dots + (y_n - \hat{y_n} )^2 \right]
$$
где $y_i$ - значения из тестовой выборки, $\hat y_i$ - значения, спрогнозированные моделью (для $i$-го образца.

**Среднеквадратическое отклонение** выражается как корень от среднеквадратической ошибки:
$$
\mathrm{RMSE} = \sqrt{\mathrm{MSE}}
$$
Функция `mean_squared_error()` вычисляет среднеквадратическую ошибку для двух массивов. Если задать параметр `squared=False`, то функция вычислит среднеквадратическое отклонение. Для наглядности, вычислим среднеквадратическую ошибку средствами NumPy, чтобы лучше представить себе, что именно происходит внутри функции `mean_squared_error()`:

In [ ]:
test_mse = np.mean((y_test - y_test_predicted) ** 2)
test_rmse = np.sqrt(test_mse)
print(test_rmse)

### Что под капотом?
Модель в машинном обучении всегда содержит внутри себя параметры, которые подгоняются к оптимальным значениям в процессе обучения. Мы выбрали модель линейной регрессии, которая реализована в классе `LinearRegression` модуля `linear_model` библиотеки `sklearn`. В результате обучения этой модели на одномерных данных, модель создает прямую $f(x) = a + bx$, которая выражает зависимость целевого признака (target feature) от единственного (нецелевого) признака. В нашем случае эта прямая должна отражать зависимость продаж мороженного (ice_cream_sales) от уличной температуры (temperature). Параметрами модели являются коэффициенты $a$ и $b$, которые однозначно задают прямую на плоскости признаков.

В случае задачи одномерной линейной регрессии "подогнать параметры к оптимальным значениям" означает найти такие $a$ и $b$, при которых прямая $f(x) = a + bx$ лучше всего выражает зависимость продаж мороженного от температуры. Но как мы будем определять, какая прямая "лучше"? Рассмотрим пару вариантов. Значение температуры произвольной $i$-строки в данных обозначим $x_i$, а соответствующее ему значение продаж мороженного обозначим $y_i$. Когда модель будет прогнозировать значение продаж мороженного по заданному значению температуры $x_i$, результатом будет $\hat y_i = a + bx_i$. Присмотримся к разности $y_i - \hat y_i$ (которую в литературе называют невязкой). Если точка оказывается выше прямой, то это значит, что $y_i > \hat y_i$ и разность будет положительной. Если точка ниже прямой, то разность отрицательная, а если точка лежит на прямой, то разность равна нулю. Что если мы возьмем среднее разностей по всем образцам наших данных и будем считать, что лучшая прямая та, у которой минимальна средняя разность?
$$
\frac{1}{n}\left[(y_1 - \hat y_1) + \dots + (y_n - \hat y_n) \right]
$$
Рассмотрим три прямые и соответствующие им средние разности

In [ ]:
def mean_differences(y1, y2):
    return np.sum(y1 - y2) / len(y1)

def f(x, a, b):
    return a + b*x

y_cross = f(X_train, 349.164, -9)
y_over = f(X_train, 200, 3)

print("mean differences")
print(f"green: {mean_differences(y_train, y_train_predicted+.01) :3f}")
print(f"orange: {mean_differences(y_train, y_cross) :3f}")
print(f"red: {mean_differences(y_train, y_over)}")

plt.scatter(X_train, y_train)
plt.plot(X_train, y_train_predicted+0.01, color='green')
plt.plot(X_train, y_cross, color='orange')
plt.plot(X_train, y_over, color='red')
# plt.ylim(0, 260)

Очевидно, что выбор прямой с минимальной средней разностью будет не лучшим решением, ведь в нашем случае это красная прямая. Это произошло от того, что все точки находятся под красной кривой и их разности отрицательные. Так, суммируя разности мы получили отрицательное число с большим абсолютным значением.

Заметим, что средняя разность оранжевой прямой, хотя она совсем не выражает зависимость в данных, довольно близка к нулю, как и средняя разность зеленой прямой. Посмотрим, какие значения принимают разности для оранжевой прямой.

In [ ]:
print((y_train - y_cross)[:5])      # первые 5 образцов
print((y_train - y_cross)[-5:])     # последние 5 образцов

Часть точек оказалось под оранжевой прямой и их разности отрицательны, а другая часть оказалась над прямой, их разности положительны. И несмотря на то, что, абсолютные значения этих разностей весьма велики, при суммировании, они компенсируют друг друга и итоговое значение средней разности оказывается близким к нулю. 

Если бы мы суммировали не сами разности, а их абсолютные значения, то такой компенсации не получилось бы, и оранжевая прямая дала бы большое значение разности. Обозначим среднее от модулей разностей буквой $E_1$
$$
E_1 = \frac{1}{n} \sum_{i=1}^{n} |y_i - \hat y_i| = \frac{1}{n}\left(|y_1 - \hat y_1| + \dots + |y_n - \hat y_n| \right)
$$

In [ ]:
def mean_abs_differences(y1, y2):
    return np.sum(np.abs(y1 - y2)) / len(y1)

print("mean abs differences")
print(f"green: {mean_abs_differences(y_train, y_train_predicted+.01) :3f}")
print(f"orange: {mean_abs_differences(y_train, y_cross) :3f}")
print(f"red: {mean_abs_differences(y_train, y_over)}")

plt.scatter(X_train, y_train)
plt.plot(X_train, y_train_predicted+0.01, color='green')
plt.plot(X_train, y_cross, color='orange')
plt.plot(X_train, y_over, color='red')
# plt.ylim(0, 260)

Минимальное значение функции $E_1$ достигается для зеленой прямой. В идеальном случае все точки лежали бы на прямой, и функция $E_1$ приняла бы значение 0. Это минимальное допустимое значение, так как сумма неотрицательных чисел не может быть меньше нуля. Чем больше точек отклоняются от прогнозируемых значений и чем сильнее они отклоняются, тем больше будет $E_1$. Поэтому можно сказать, что функция $E_1$ характеризует ошибочность модели на данных. Чем меньше будет $E_1$, тем меньше ее ошибочность, и, соответственно, лучше результаты. 

### Метод наименьших квадратов
Метод наименьших квадратов представляет собой алгоритм машинного обучения, который позволяет решать задачу линейной регрессии. В этом методе параметры модели подгоняются так, чтобы среднее значение квадратов разностей $y_i - \hat y_i$ оказалось минимальным.
$$
E_2 = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat y_i)^2 = \frac{1}{n}\left[(y_1 - \hat y_1)^2 + \dots + (y_n - \hat y_n)^2 \right]
$$
В случае одномерной регрессии минимальное значение функции $E_2$ достигается при следующих значениях параметров $a$ и $b$:
$$\hat a = \overline y - \hat b \overline x$$
$$\hat b = \frac{ \sum_{i=1}^n (x_i - \overline x)(y_i - \overline y)}{\sum_{i=1}^n (x_i - \overline x)^2}
$$
где $ \overline x$ и  $\overline y$ - это средние значения.

Именно таким образом вычисляются оптимальные параметры модели класса `LinearRegression` при вызове метода `fit()`. Получить коэффициенты $a$ и $b$ из модели можно через свойства `intercept_` и `coef_` соответственно:

In [ ]:
a = model.intercept_
b = model.coef_
print(f'{a = }, {b = }')

При вызове метода `predict()` модель вычисляет прогнозируемое значение для каждого $i$-го образца из `X_test` по формуле $\hat y_i = a + bx_i$. Убедимся в этом самостоятельно вычислив значения $\hat y_i$ воспользовавшись полученными из модели параметрами `a` и `b`:

In [ ]:
y = a + b * X_train
# y = model.predict(X_train.reshape(-1, 1))

plt.scatter(X_train, y_train, c='grey', s=15)
plt.scatter(X_test, y_test, c='green')
plt.plot(X_train, y)
# plt.plot(X_train, y_train_predicted)
plt.ylim(0, 270)
plt.xlabel('temperature')
plt.ylabel('ice_cream_sales')

### Разбиение данных на обучающую и тестовую выборки 

In [ ]:
from sklearn.model_selection import train_test_split

data = pd.read_csv('ice_cream_selling_train.csv')

train_set, test_set = train_test_split(data, test_size=0.2, random_state=42)
print(type(train_set))  # типы данных DataFrame
print(len(train_set), len(test_set))

In [ ]:
test_set

In [ ]:
train_set